go over pytorch tutorial from docs


In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import sys

In [12]:
# download training data from domain

training_data = datasets.FashionMNIST(
    root='data/',
    train=True,
    download=True,
    transform=ToTensor(),
)

testing_data = datasets.FashionMNIST(
    root='data/',
    train=False,
    download=True,
    transform=ToTensor(),
)

In [13]:
batch_size= 64

# create data loaders; used to make our datasets iterable of batch size
train_dataloader = DataLoader(training_data, batch_size)
test_dataloader = DataLoader(testing_data, batch_size)

for X, y in test_dataloader:
    print('Shape of X [N, C, H, W]:', X.shape)
    print('shape of y: ', y.shape, y.dtype)
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y:  torch.Size([64]) torch.int64


In [14]:
print(sys.getrecursionlimit())
sys.setrecursionlimit(100000)
print(sys.getrecursionlimit())

100000
100000


In [25]:
# creat models
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
# print("Using {} device".format(device))
# define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [16]:
# optmization and loss fn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [22]:
# make functions for running nn on our train and test
# in the loop, teh model makes predictions from the training data (and each iteration will
# be a batch of data), after predictions we backpropagate the prediction erros to adjust
# the parameters

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # computer pred error
        pred = model(X)
        loss = loss_fn(pred, y)

        # bp
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [18]:
# can also check the models performacne against the dataset to ensure it is learning
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= size
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [26]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299259  [    0/60000]
loss: 2.299857  [ 6400/60000]
loss: 2.304546  [12800/60000]
loss: 2.311232  [19200/60000]
loss: 2.295558  [25600/60000]
loss: 2.308534  [32000/60000]
loss: 2.303639  [38400/60000]
loss: 2.303327  [44800/60000]
loss: 2.301146  [51200/60000]
loss: 2.308671  [57600/60000]
Test Error: 
 Accuracy: 15.2%, Avg loss: 0.036169 

Epoch 2
-------------------------------
loss: 2.299259  [    0/60000]
loss: 2.299857  [ 6400/60000]
loss: 2.304546  [12800/60000]
loss: 2.311232  [19200/60000]
loss: 2.295558  [25600/60000]
loss: 2.308534  [32000/60000]
loss: 2.303639  [38400/60000]
loss: 2.303327  [44800/60000]
loss: 2.301146  [51200/60000]
loss: 2.308671  [57600/60000]
Test Error: 
 Accuracy: 15.2%, Avg loss: 0.036169 

Epoch 3
-------------------------------
loss: 2.299259  [    0/60000]
loss: 2.299857  [ 6400/60000]
loss: 2.304546  [12800/60000]
loss: 2.311232  [19200/60000]
loss: 2.295558  [25600/60000]
loss: 2.308534  [32000/600

In [ ]:
# save model

# torch.save(model.state_dict(), 'models/tutorial_model')

In [ ]:
# loading models

# tmp = NeuralNetwork()
# tmp.load_state_dict(torch.load('models/tutorial_model'))